<a href="https://colab.research.google.com/github/ompatil-tech/IoT-Anomaly-Detection-ML/blob/main/Final_Model_Dynamic_Neuro_Symbolic_Agent_for_Enhanced_Arithmetic_Reasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================================
# DIRECT DOWNLOAD BY ID
# ==========================================

# 1. INSTALL LIBRARIES
import subprocess
print("🛠️ Installing libraries...")
subprocess.run("pip install -q transformers peft accelerate bitsandbytes gdown", shell=True)

import os
import torch
import json
import re
import ast
from dataclasses import dataclass
from typing import Any, Callable, Dict
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# ---------------------------------------------------------
# 2. DOWNLOAD FILES DIRECTLY (Bypassing Drive Mounting)
# ---------------------------------------------------------
MODEL_DIR = "my_fine_tuned_model"
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

print(f"⬇️ Downloading model files into '{MODEL_DIR}'...")

# File 1: adapter_config.json
# ID: 1WXM4hHkIjMG46dkEA7hzl5mqRva2k3YF
subprocess.run(f"gdown 1WXM4hHkIjMG46dkEA7hzl5mqRva2k3YF -O {MODEL_DIR}/adapter_config.json", shell=True)

# File 2: adapter_model.safetensors
# ID: 1HJ3SINyzfO8A64BlIK6Pav3uCuNQt-Q4
subprocess.run(f"gdown 1HJ3SINyzfO8A64BlIK6Pav3uCuNQt-Q4 -O {MODEL_DIR}/adapter_model.safetensors", shell=True)

# Verify download
if os.path.exists(f"{MODEL_DIR}/adapter_config.json") and os.path.exists(f"{MODEL_DIR}/adapter_model.safetensors"):
    print("✅ Files downloaded successfully!")
else:
    raise FileNotFoundError("❌ Download failed. Google might be blocking the download due to high traffic. Try again in 1 minute.")

# ---------------------------------------------------------
# 3. DEFINE AGENT & TOOLS
# ---------------------------------------------------------
def extract_json_object(text: str) -> Dict[str, Any]:
    s = text.strip()
    s = re.sub(r"^```[a-zA-Z]*", "", s)
    s = re.sub(r"```$", "", s).strip()
    match_list = re.search(r'\[\s*\{.*?\}\s*\]', s, re.DOTALL)
    if match_list:
        try:
            data = json.loads(match_list.group(0))
            if isinstance(data, list) and len(data) > 0: return data[0]
        except: pass
    match_obj = re.search(r"\{.*\}", s, re.DOTALL)
    if match_obj:
        try: return json.loads(match_obj.group(0))
        except: pass
    raise ValueError(f"No JSON found in: {text[:20]}...")

@dataclass
class Tool:
    name: str
    description: str
    func: Callable

def calculator_func(args):
    expr = str(args.get("expression", ""))
    clean_expr = re.sub(r'(\d),(\d)', r'\1\2', expr)
    try:
        allowed = set("0123456789+-*/(). ")
        if not set(clean_expr).issubset(allowed): return {"error": "Unsafe chars"}
        return {"result": eval(clean_expr)}
    except Exception as e:
        return {"error": str(e)}

calc_tool = Tool("calculator", "Math evaluator", calculator_func)

class ToolAugmentedLlama:
    def __init__(self, model, tokenizer, tools):
        self.model = model
        self.tokenizer = tokenizer
        self.tools = {t.name: t for t in tools}

    def generate(self, user_input):
        # ROBOT PROMPT
        system_prompt = """[SYSTEM: CODE MODE]
You are a JSON generator. You DO NOT chat.
Output ONLY a JSON object for the tool.
FORMAT: {"tool": "calculator", "args": {"expression": "100+50"}}
"""
        prompt = f"[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_input} [/INST]"
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        with torch.no_grad():
            outputs = self.model.generate(**inputs, max_new_tokens=128, do_sample=False)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        try:
            tool_call = extract_json_object(response)
            if tool_call["tool"] in self.tools:
                result = self.tools[tool_call["tool"]].func(tool_call["args"])
                return {"final_answer": f"Calculated: {result}", "tool_calls": [tool_call]}
        except: pass
        return {"final_answer": response, "tool_calls": []}

# ---------------------------------------------------------
# 4. LOAD MODEL (Your Fine-Tuned Version)
# ---------------------------------------------------------
print(f"🔄 Loading Model from {MODEL_DIR}...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
base_model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-2-7b-chat-hf",
    quantization_config=bnb_config,
    device_map="auto"
)
# Load Adapter from the specific download folder
model = PeftModel.from_pretrained(base_model, MODEL_DIR)
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
print("✅ Model Loaded!")

# ---------------------------------------------------------
# 5. RUN TEST
#

🛠️ Installing libraries...
⬇️ Downloading model files into 'my_fine_tuned_model'...
✅ Files downloaded successfully!
🔄 Loading Model from my_fine_tuned_model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model Loaded!


In [80]:
# ==========================================
# FINAL ROBUST AGENT: IMPROVED LOGIC EXAMPLES
# ==========================================
import json
import re
import torch

class ToolAugmentedLlama:
    def __init__(self, model, tokenizer, tools):
        self.model = model
        self.tokenizer = tokenizer
        self.tools = {t.name: t for t in tools}

    def generate(self, user_input):
        # 1. IMPROVED PROMPT (TEACHING THE LOGIC)
        # We give examples that show DEPENDENCIES (C depends on B).
        prompt = f"""You are a calculator. Extract numbers and calculate sum.

Example 1 (Simple):
Input: The Apple is 5. The Pear is 10. The Banana is 20. Calculate sum.
Output: {{"tool": "calculator", "args": {{"expression": "5 + 10 + 20"}}}}

Example 2 (Complex Dependencies):
Input: Box A is 100. Box B is double Box A. Box C is 50 less than Box B.
Output: {{"tool": "calculator", "args": {{"expression": "100 + (100*2) + ((100*2)-50)"}}}}

Example 3 (Mixed):
Input: The Shirt is 20. Pants are double the Shirt. Hat is 5 more than Pants.
Output: {{"tool": "calculator", "args": {{"expression": "20 + (20*2) + ((20*2)+5)"}}}}

Task:
Input: {user_input}
Output: {{"tool": "calculator", "args": {{"expression": "
"""

        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        with self.model.disable_adapter():
            with torch.no_grad():
                outputs = self.model.generate(**inputs, max_new_tokens=128, do_sample=False)

        full_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # 2. PARSING
        try:
            # Grab everything after the prompt ends
            raw_math = full_text.split('expression": "')[-1]

            # Trim whitespace and newlines
            clean_math = raw_math.strip()

            # Stop at the first quote or newline
            if '"' in clean_math:
                clean_math = clean_math.split('"')[0]
            else:
                clean_math = clean_math.split('\n')[0]

            # Reconstruct valid JSON
            json_reconstructed = '{"tool": "calculator", "args": {"expression": "' + clean_math + '"}}'

            tool_call = json.loads(json_reconstructed)
            if "tool" in tool_call and tool_call["tool"] in self.tools:
                result = self.tools[tool_call["tool"]].func(tool_call["args"])
                return {
                    "final_answer": f"Calculated Result: {result}",
                    "tool_calls": [tool_call]
                }
        except Exception as e:
             return {
                 "final_answer": f"Fix failed: {e}",
                 "raw_output": full_text,
                 "tool_calls": []
             }

        return {"final_answer": "Failed", "raw_output": full_text, "tool_calls": []}

# Re-Initialize
agent = ToolAugmentedLlama(model, tokenizer, [calc_tool])

print("🤖 RUNNING COMPLEX LOGIC TEST...")
test_prompt = """The Pen costs 5 dollars. The Notebook costs triple that amount. The Highlighter costs 2 dollars. Calculate the total cost. """

result = agent.generate(test_prompt)

print("\n=== FINAL RESULT ===")
print(result["final_answer"])
print("\n=== RAW TOOL CALL ===")
if "tool_calls" in result and result["tool_calls"]:
    print(json.dumps(result["tool_calls"], indent=2))

🤖 RUNNING COMPLEX LOGIC TEST...

=== FINAL RESULT ===
Calculated Result: {'result': 22}

=== RAW TOOL CALL ===
[
  {
    "tool": "calculator",
    "args": {
      "expression": "5 + (3*5) + 2"
    }
  }
]


In [81]:
# ==========================================
# 6. LAUNCH INTERACTIVE UI (Gradio)
# ==========================================
import gradio as gr
import json
import subprocess

# 1. Install Gradio (if not already installed)
try:
    import gradio
except ImportError:
    print("🛠️ Installing Gradio UI...")
    subprocess.run("pip install -q gradio", shell=True)
    import gradio as gr

# 2. Define the Wrapper Function
def solve_puzzle(user_text):
    """
    This function connects the Web UI inputs to your Agent's generate method.
    """
    if not user_text.strip():
        return "Please enter a puzzle.", "{}"

    # Run the Agent (The agent object 'agent' must be loaded from your previous cell)
    try:
        response = agent.generate(user_text)

        # Format the output for the user
        answer = response.get("final_answer", "No final answer generated.")

        # Format the tool logs for debugging/grading
        tools_used = response.get("tool_calls", [])
        tool_log = json.dumps(tools_used, indent=2) if tools_used else "No tools used (check final answer for raw model output)."

        return answer, tool_log

    except Exception as e:
        return f"Error processing request: {str(e)}", "{}"

# 3. Create the Web Interface
demo = gr.Interface(
    fn=solve_puzzle,
    inputs=gr.Textbox(
        lines=5,
        placeholder="Type your logic puzzle here...\nExample: Tank A has 40 fish. Tank B has triple that amount. Tank C has 20 fish. Calculate the total number of fish.",
        label="Your Logic Puzzle"
    ),
    outputs=[
        gr.Textbox(label="Agent Solution (Calculated Result)"),
        gr.Code(label="Tool Logic (For Grading)", language="json")
    ],
    title="Neuro-Symbolic Agent Tester",
    description="Enter a word problem. The Agent uses Llama-2-7B to extract the logic and a Python calculator to find the numerical result. ",
    theme="soft"
)

# 4. Launch!
print("🚀 Launching UI... Click the public link below to access the interface!")
demo.launch(share=True, debug=True)

🚀 Launching UI... Click the public link below to access the interface!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://305ef4c52ec25952a4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://305ef4c52ec25952a4.gradio.live
